# Order Brushing Detection

## Part of Shopee Code League Challenge (Day 1)

Well, shopee finally conducted its code league event first time online due to the corona virus. This decision will only bring them any good since the antusiasm of the participants from 5 countries in Asean dramatically increased. Probably because it becomes much easier for participants to join the competion from the comfort of their own houses. I also took part as one of the participants along with my colleague.

The event itself is not absent / is not empty from several technical difficulties. At first, due to the high rate if participants that clicked on the kaggle link at the same time, the site was immediately crashed. Fortunately, the problems could be resolved later despite a considerable amount of delay that was experienced by participants. But, from what I saw, it did not shrink the spirit of all participants.

This is the first time I experienced this kind of challenge. I found this kind of problem good for practice because It really examine / challenge our ability to think computationally / logically combined with our data understanding and our comprehension of the task itself. We have to first, what I found to be the most important part, understand the task and know how the output should look like, because it will affect our score at the end. Perhaps, for someone that do loop gymnastic on day to day basis, this kind of problem is so easy to comprehend amd answer. But for those with zero experience in advanced looping, and always rely on tools and package in python to solve the frequent data problem, it will be such an interesting challenge. 

The challenge is called order brushing. Basically, it asks us to detect abnormal user behaviour from their e-commerce platform. For instance, if one seller is categorized as best seller, but after thorough examination from the record, we find that the orders came from one user repeatedly, we have evidence to suspect that that particular shop conduct order brushing practice. What makes this practice a bad practice is because the order that is conducted or made by that same user is not a real order, but it is a fake one to level up / increase the rating of the shop. For this practice, usually every e-commerce must be aware of it and make some system such as flagging, as what shopee team did currently. 

For the task, I will quote exactly with what is written in competion's Kaggle page:

1. Identify all shops that are deemed to have conducted order brushing.
2. For each shop that is identified to have conducted order brushing, identify the buyers suspected to have conducted order brushing for that shop.

For the purpose of this question, shops are deemed to have conducted order brushing if their concentrate rate is greater than or equal to 3 at any instance

Concentrate rate = 
Number of Orders within 1 hour / Number of Unique Buyers within 1 hour
For the purpose of this question, suspicious buyers are deemed as the buyer that contributed the highest proportion of orders to a shop that is deemed to have conducted order brushing.

For calculation of the highest proportion of orders to a shop, only include the orders that occured in instances when order brushing has been deemed to have taken place.

In the case where multiple users share the same highest proportion of orders for a specific shop, all those users are deemed to be suspicious buyers.

So, let's try to tackle this challenge as much as we can do!

Firstly, we only need two packages, Pandas and NumPy, to deal with this problem. Pandas and NumPy are more than enough to help us with looping and solving list problem.

In [2]:
import pandas as pd
import numpy as np

Fortunately, the data is already in csv format, so we don't have to do scrapping and crawling from the web. We read this data directly and import it to the notebook.

In [89]:
order_brush = pd.read_csv("order_brush_order.csv")

Let's see first ten rows of our table.

In [4]:
order_brush.head(10)

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
5,31119725718155,173811070,116451780,2019-12-27 12:22:05
6,31151322178251,107921853,166741763,2019-12-27 21:08:43
7,31079856153738,178400128,61272835,2019-12-27 01:17:37
8,31118426867571,147941492,10986763,2019-12-27 12:00:27
9,31133458226149,164933170,135957741,2019-12-27 16:10:59


So, we have in our hand four important columns that we should determine to understand which orders that are suspicious for order brushing practice. 

In order to know the data type and condition of the table, whether it consist of missing data or not, we can use this method below.

In [133]:
order_brush.isnull().sum()

orderid       0
shopid        0
userid        0
event_time    0
dtype: int64

The data is really clean, no missing data found, so we will keep it as it is without cleaning and manipulating.

In [134]:
order_brush.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
orderid       222750 non-null int64
shopid        222750 non-null int64
userid        222750 non-null int64
event_time    222750 non-null object
dtypes: int64(3), object(1)
memory usage: 6.8+ MB


All columns is in correct data type unless event_time column that suppose to be a datetime, but we will deal with it later.

It will be useful if we know how much the number of order and number of seller recorded in this table as shown below.

In [91]:
num_order = order_brush.shape[0]
num_seller = len(order_brush['shopid'].unique())

print(num_order)
print(num_seller)

222750
18770


This is the part where we start to tackle the challenge. This challenge require us to count the number of orderid and userid of each shop in this record, but it will be difficult if we use the original table, even thought we already group it by shopid. In this regard, we will be utilizing the list function / list data structure to store all orderid and userid that belongs to particular shop. In this way, it would be easier for us to count and play with the data that is already stored in list. 

To implement this, first, we group the original table by shopid, since we want to examine each shop individually. To store all orderid that belongs to shopid in the list, we apply function list to orderid column and convert it to dataframe as shown below. We assign this new table to variable named ls_order.

In [92]:
ls_order = order_brush.groupby('shopid')['orderid'].apply(list).to_frame()

We also store userid in the new list by applying list function to userid column in original table and convert it to dataframe. New column is created to store this listed userid in the new table that we created previously.

In [93]:
ls_order['userid'] = order_brush.groupby('shopid')['userid'].apply(list).to_frame()

We also create new column called event_time inside ls_order table by applying list function to event_time from original table and convert it to dataframe.

In [95]:
ls_order['event_time'] = order_brush.groupby('shopid')['event_time'].apply(list).to_frame()

Let's check our new table that consists four column with listed values.

In [96]:
ls_order

,orderid,userid,event_time
shopid,,,
10009,[31086409141107],[196962305],[2019-12-27 03:06:50]
10051,"[31254979546679, 31144571933461]","[48600461, 2854032]","[2019-12-29 01:56:19, 2019-12-27 19:16:11]"
10061,"[31431527100615, 31205132327893, 3119567591920...","[62464559, 194819216, 168750452, 130633421]","[2019-12-31 02:58:48, 2019-12-28 12:05:32, 201..."
10084,"[31446282541417, 31377753982301, 3126554273178...","[123715078, 184840040, 6488128, 73993513, 3328...","[2019-12-31 07:04:42, 2019-12-30 12:02:33, 201..."
10100,"[31219662667629, 31415528979818, 3147112150853...","[495431, 30534780, 136619968, 198765759, 13146...","[2019-12-28 16:07:42, 2019-12-30 22:32:08, 201..."
...,...,...,...
214662358,[31420157746353],[194332514],[2019-12-30 23:49:17]
214949521,"[31493203163305, 31491997643888]","[46269178, 46269178]","[2019-12-31 20:06:43, 2019-12-31 19:46:37]"
214964814,[31328775676314],[200983383],[2019-12-29 22:26:16]


To know the number of order from each shop, we then count the number of values that is stored in orderid column. Since we have stored all the values in one list, it will be much easier for us to count them. We use len() method to know how many values stored in a list. With the power of map and lambda we apply len() function to all rows recorded in this table. We store the number of orders in new column called num_order.

In [97]:
ls_order["num_order"] = list(map(lambda x: len(x), ls_order.orderid))

Now, we got the number of orders that is stored in new column.

In [98]:
ls_order

,orderid,userid,event_time,num_order
shopid,,,,
10009,[31086409141107],[196962305],[2019-12-27 03:06:50],1
10051,"[31254979546679, 31144571933461]","[48600461, 2854032]","[2019-12-29 01:56:19, 2019-12-27 19:16:11]",2
10061,"[31431527100615, 31205132327893, 3119567591920...","[62464559, 194819216, 168750452, 130633421]","[2019-12-31 02:58:48, 2019-12-28 12:05:32, 201...",4
10084,"[31446282541417, 31377753982301, 3126554273178...","[123715078, 184840040, 6488128, 73993513, 3328...","[2019-12-31 07:04:42, 2019-12-30 12:02:33, 201...",55
10100,"[31219662667629, 31415528979818, 3147112150853...","[495431, 30534780, 136619968, 198765759, 13146...","[2019-12-28 16:07:42, 2019-12-30 22:32:08, 201...",42
...,...,...,...,...
214662358,[31420157746353],[194332514],[2019-12-30 23:49:17],1
214949521,"[31493203163305, 31491997643888]","[46269178, 46269178]","[2019-12-31 20:06:43, 2019-12-31 19:46:37]",2
214964814,[31328775676314],[200983383],[2019-12-29 22:26:16],1


After we know number of order of each shop, in order to detect order brushing, we must also know which buyers make an order repeatedly out of those number of order. Hence, we should know first from those number of order, how much actually the number of buyer that is existed. Does each order belong to one buyer or each order belong to many buyer, which could be an evidence for order brushing. With that in mind, we have to know the unique value from orderid by using nunique() function, and store it in new column called num_unique_buyer.

In [99]:
ls_order['num_unique_buyer'] = order_brush.groupby('shopid')['userid'].nunique()

In [100]:
ls_order

,orderid,userid,event_time,num_order,num_unique_buyer
shopid,,,,,
10009,[31086409141107],[196962305],[2019-12-27 03:06:50],1,1
10051,"[31254979546679, 31144571933461]","[48600461, 2854032]","[2019-12-29 01:56:19, 2019-12-27 19:16:11]",2,2
10061,"[31431527100615, 31205132327893, 3119567591920...","[62464559, 194819216, 168750452, 130633421]","[2019-12-31 02:58:48, 2019-12-28 12:05:32, 201...",4,4
10084,"[31446282541417, 31377753982301, 3126554273178...","[123715078, 184840040, 6488128, 73993513, 3328...","[2019-12-31 07:04:42, 2019-12-30 12:02:33, 201...",55,54
10100,"[31219662667629, 31415528979818, 3147112150853...","[495431, 30534780, 136619968, 198765759, 13146...","[2019-12-28 16:07:42, 2019-12-30 22:32:08, 201...",42,39
...,...,...,...,...,...
214662358,[31420157746353],[194332514],[2019-12-30 23:49:17],1,1
214949521,"[31493203163305, 31491997643888]","[46269178, 46269178]","[2019-12-31 20:06:43, 2019-12-31 19:46:37]",2,1
214964814,[31328775676314],[200983383],[2019-12-29 22:26:16],1,1


Great! we now get the number of unique buyer from each of shopid. We could filter the table based on this value since order brushing only happen when number of unique buyer is more or equal than 2. We filter them to exclude all of the shopid that has same number both from orderid and number of unique buyer.

In [101]:
# Order brushing might happens if the number of orders per shop minus number of unique buyers per shop is larger or equal to 2.

ls_order = ls_order[ls_order['num_order']-ls_order['num_unique_buyer'] >= 2]

We check the current condition of our table. After that filtering, we only got approximately 10% of original table that fulfill/meet the condition of order brushing. That is a major exclusion/filtering. Apparently, the number of shop who conduct this bad order brushing practice is only a small amount of all shop that is recorded in this table, however, no matter how small it is, it still has to be treated seriously.

We will be working with this new table from now on.

In [102]:
ls_order

,orderid,userid,event_time,num_order,num_unique_buyer
shopid,,,,,
10100,"[31219662667629, 31415528979818, 3147112150853...","[495431, 30534780, 136619968, 198765759, 13146...","[2019-12-28 16:07:42, 2019-12-30 22:32:08, 201...",42,39
10151,"[31414025664718, 31255231246682, 3112887694712...","[214925713, 43528657, 36268816, 21575761, 8614...","[2019-12-30 22:07:05, 2019-12-29 02:00:32, 201...",193,188
10159,"[31425048460235, 31458346123017, 3133134606985...","[24869430, 802231, 136820136, 125911190, 10581...","[2019-12-31 01:10:48, 2019-12-31 10:25:47, 201...",99,91
10208,"[31117173986024, 31320198136784, 3137988177059...","[86369812, 91720728, 78503053, 211633164, 7991...","[2019-12-27 11:39:33, 2019-12-29 20:03:18, 201...",182,169
10214,"[31293579591969, 31294622928885, 3112234450768...","[89600015, 112947907, 160166910, 122964607, 21...","[2019-12-29 12:39:39, 2019-12-29 12:57:02, 201...",73,69
...,...,...,...,...,...
211245313,"[31477256052036, 31228328416550, 3133618963726...","[26716334, 63112942, 2048831, 16871221, 428273...","[2019-12-31 15:40:57, 2019-12-28 18:32:08, 201...",52,45
211892380,"[31089143023034, 31242108028092, 3141262897924...","[214237579, 10178183, 40838578, 162920162, 817...","[2019-12-27 03:52:24, 2019-12-28 22:21:48, 201...",13,11
212089630,"[31451185029030, 31454849817709, 31450924483632]","[105664365, 105664365, 105664365]","[2019-12-31 08:26:26, 2019-12-31 09:27:30, 201...",3,1


Now, we come to the part where it becomes more challenging and fun obviously. We have to create custom function in order to extract the suspicious buyer from userid that has been filtered previously. The name of the function is defined as pick with alist as an input. Firstly, we create empty list named a to store the result in it. Then, we create for loop to the alist column, which in this regard will be userid, with condition if the number of values in userid list is more or equal than 3, store those values (userid) in empty list a. After looping has been done, if empty list a still got no values, return zero as a result. If there are some values in variable a, then return those list after sorting them.

In [103]:
# identify suspicious buyer using custom function

def pick(alist):
    a = []
    # for values in the row in column/alist, in this regard it will be used for "userid"
    for i in alist:
        # if the number of values in list in "userid" column greater / equal than 3
        if alist.count(i) >= 3:
            # store user id in empty list a
            a.append(i)
    # if there is no values in list a after looping, return 0 as a result
    if a == []:
        return 0
    else:
        # otherwise, return list packed with values that has been sorted before
        return sorted(list(set(a)))
    


C:\Users\Faris\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


The function above is applied toward the userid column and store the result of these function in new column called suspicious_buyer.

In [ ]:
ls_order['suspicious_buyer'] = list(map(pick, ls_order['userid']))

We got the orderid which becomes candidate for order brushing practice in the form of list. Let's see how our table looks like now.

In [104]:
ls_order

,orderid,userid,event_time,num_order,num_unique_buyer,suspicious_buyer
shopid,,,,,,
10100,"[31219662667629, 31415528979818, 3147112150853...","[495431, 30534780, 136619968, 198765759, 13146...","[2019-12-28 16:07:42, 2019-12-30 22:32:08, 201...",42,39,0
10151,"[31414025664718, 31255231246682, 3112887694712...","[214925713, 43528657, 36268816, 21575761, 8614...","[2019-12-30 22:07:05, 2019-12-29 02:00:32, 201...",193,188,0
10159,"[31425048460235, 31458346123017, 3133134606985...","[24869430, 802231, 136820136, 125911190, 10581...","[2019-12-31 01:10:48, 2019-12-31 10:25:47, 201...",99,91,"[62775725, 214988798]"
10208,"[31117173986024, 31320198136784, 3137988177059...","[86369812, 91720728, 78503053, 211633164, 7991...","[2019-12-27 11:39:33, 2019-12-29 20:03:18, 201...",182,169,[11288307]
10214,"[31293579591969, 31294622928885, 3112234450768...","[89600015, 112947907, 160166910, 122964607, 21...","[2019-12-29 12:39:39, 2019-12-29 12:57:02, 201...",73,69,0
...,...,...,...,...,...,...
211245313,"[31477256052036, 31228328416550, 3133618963726...","[26716334, 63112942, 2048831, 16871221, 428273...","[2019-12-31 15:40:57, 2019-12-28 18:32:08, 201...",52,45,[46875705]
211892380,"[31089143023034, 31242108028092, 3141262897924...","[214237579, 10178183, 40838578, 162920162, 817...","[2019-12-27 03:52:24, 2019-12-28 22:21:48, 201...",13,11,0
212089630,"[31451185029030, 31454849817709, 31450924483632]","[105664365, 105664365, 105664365]","[2019-12-31 08:26:26, 2019-12-31 09:27:30, 201...",3,1,[105664365]


Since we also include zero as one of the result in suspicious_buyer, we will filter out / exclude those zero so that we know how much is the shopid that is on target for order brushing practice. We create new variable called susp_order to store this new table.

In [105]:
# identify suspicious order brushing

susp_order = ls_order[ls_order['suspicious_buyer'] != 0]

If we look at the table below, there is nearly 50% that have been filtered out from previous table. Now, we only got 828 rows that is suspicious. The number of shopid shrinks as we go through this step to find the shops that are responsible for order brushing practice.

In [106]:
susp_order

,orderid,userid,event_time,num_order,num_unique_buyer,suspicious_buyer
shopid,,,,,,
10159,"[31425048460235, 31458346123017, 3133134606985...","[24869430, 802231, 136820136, 125911190, 10581...","[2019-12-31 01:10:48, 2019-12-31 10:25:47, 201...",99,91,"[62775725, 214988798]"
10208,"[31117173986024, 31320198136784, 3137988177059...","[86369812, 91720728, 78503053, 211633164, 7991...","[2019-12-27 11:39:33, 2019-12-29 20:03:18, 201...",182,169,[11288307]
10287,"[31199080184075, 31160740703944, 3109326538833...","[495270, 875954, 101436986, 198097381, 1545368...","[2019-12-28 10:24:41, 2019-12-27 23:45:40, 201...",211,190,"[74488711, 103289694, 198097381, 214226569]"
10402,"[31471217004140, 31075350383695, 3146785379529...","[215413147, 185138818, 65216466, 77819, 203648...","[2019-12-31 14:00:17, 2019-12-27 00:02:30, 201...",13,11,[77819]
10536,"[31193005938624, 31115953007150, 3113916044439...","[153387696, 672345, 672345, 672345, 672345, 67...","[2019-12-28 08:43:25, 2019-12-27 11:19:13, 201...",13,5,[672345]
...,...,...,...,...,...,...
209326384,"[31491123051077, 31483532950205, 3133251414481...","[7794832, 7794832, 88887227, 7794832]","[2019-12-31 19:32:04, 2019-12-31 17:25:32, 201...",4,2,[7794832]
210197928,"[31103266871471, 31103396827994, 3110333467196...","[52867898, 52867898, 52867898, 52867898]","[2019-12-27 07:47:47, 2019-12-27 07:49:56, 201...",4,1,[52867898]
211245313,"[31477256052036, 31228328416550, 3133618963726...","[26716334, 63112942, 2048831, 16871221, 428273...","[2019-12-31 15:40:57, 2019-12-28 18:32:08, 201...",52,45,[46875705]


We now create dictionary with index of ls_order table which is shopid as a key and listed suspicious_buyer as a values. This table still contains zero value in suspicious_buyer column. 

In [107]:
dict_shop_user = dict(zip(ls_order.index, ls_order.suspicious_buyer))

In [108]:
#dict_shop_user

In [137]:
dict_shop_user

{10100: 0,
 10151: 0,
 10159: [62775725, 214988798],
 10208: [11288307],
 10214: 0,
 10287: [74488711, 103289694, 198097381, 214226569],
 10311: 0,
 10402: [77819],
 10429: 0,
 10484: 0,
 10536: [672345],
 10563: [163062835],
 12568: 0,
 13511: 0,
 16107: 0,
 26133: [32449911],
 29583: [29696494, 214495220, 215275495],
 42472: [740844],
 42818: [170385453],
 45096: [68680945],
 56418: 0,
 76934: [124597967, 190449497],
 80108: 0,
 94090: [22525413],
 94636: 0,
 140266: 0,
 150722: 0,
 183068: [2973390, 107641182, 181459643],
 192924: 0,
 195531: [214992524],
 226382: 0,
 240309: [27567],
 257944: 0,
 299742: [2973390, 131375087, 144780413],
 316121: 0,
 406582: [212200633],
 425364: [72914921],
 599533: [264511],
 605561: [166678691, 181682008],
 639650: 0,
 640697: 0,
 641249: 0,
 720918: 0,
 722132: [7670129],
 731606: [75558350],
 736620: [62618064],
 739014: [99030602, 105840033],
 760726: 0,
 769445: [641121, 29299481, 141006168],
 798888: 0,
 801940: [565381, 106797270],
 803386:

Another function is defined as order_brushing_detection with shopid as an input. Firstly, we create 2 variables named users and fraud. For users it will store the dictionary that has been created previously but now with index from susp_order table as its keys, which has been filtered out from zero values, in other word, this is the value that we suspect. For fraud, it will store the userid that is categorized as conducting order brushing. We still have not worked with the event_time column before, so we create list_time variables inside our first "users" for loop to filter the time that belongs to the suspicious shopid and orderid in a form of list. Second loop is created toward list_time to convert event_time that still in string datatype to datetime type and store it in empty variable timestamp. Third loop is made to the range of length of timestamp minus 2 to find the time difference (in seconds) between each time in those variable, and store it to diff_time variable. From the task we understand that order brushing is calculated in the basis of 3600 seconds or 1 hour period. As we got the diff_time values in the form of numpy array, we create condition that tell if the sum of diff_time is less than 3600 seconds (1 hour), then store the userid in users dictionary inside fraud variable. 

The loop is over and we got the values inside the list in fraud variable. From the task, it is said that if there are 2 or more userid in one shop, then use "&" to concat them together, so we use .join() in Python to help us with this. Last conditional statement will be if fraud is empty string, then return zero as a result. But, if it has values, then return that values as a result.

In [110]:
def order_brushing_detection(shopid):
    users = dict_shop_user[shopid]
    fraud = []
    for user in users:
        list_time = sorted(list(np.array(ls_order.loc[shopid].event_time)[np.array(ls_order.loc[shopid].userid) == user]))
        timestamp = []
        for t in list_time:
            timestamp.append(datetime.strptime(t,'%Y-%m-%d %H:%M:%S'))
        diff_time = []
        for i in range(len(timestamp)-2):
            diff_time.append((timestamp[i+2] - timestamp[i]).seconds)
        diff_time = np.array(diff_time)
        if sum(diff_time < 3600):
            fraud.append(user)
    fraud = '&'.join([str(i) for i in sorted(fraud)])
    if fraud == '':
        return 0
    else:
        return fraud

The order_brushing_det then is applied into susp_order table with its index, then convert it into list.

In [111]:
fraud_buyer = list(map(order_brushing_det, susp_order.index))

Let's see the fraud_buyer list below.

In [115]:
fraud_buyer

['62775725&214988798',
 0,
 '74488711&198097381&214226569',
 '77819',
 '672345',
 0,
 0,
 '214495220&215275495',
 '740844',
 '170385453',
 0,
 '190449497',
 '22525413',
 0,
 '214992524',
 0,
 0,
 '212200633',
 '72914921',
 '264511',
 '181682008',
 '7670129',
 '75558350',
 '62618064',
 0,
 '29299481&141006168',
 '565381',
 '188942105',
 0,
 0,
 0,
 '122277324',
 0,
 '181408876',
 0,
 0,
 '3538134',
 0,
 '15053804',
 '78451300&123959597',
 '214568881',
 0,
 '80690628',
 0,
 0,
 '9209456',
 '16339607&212325226',
 '143847348',
 0,
 0,
 0,
 0,
 '740844',
 0,
 0,
 '425354',
 '556867',
 '9753706',
 '13135622&137245836&162508227&215382704',
 '197569856',
 0,
 '139795934',
 '148176353',
 0,
 0,
 '84811421',
 '43131979',
 '210920501',
 0,
 '8405753',
 '95058664',
 '199416406',
 0,
 '152292010',
 '148215831',
 '214546342',
 '136680607',
 '148215831',
 '156614746',
 '2779333&214588488',
 '48412388',
 0,
 0,
 0,
 '128838735',
 0,
 '195918606',
 0,
 '215424202',
 0,
 '32594',
 0,
 '128702876',
 '200

For the purpose of submission, let's turn this list back into dataframe called fraud_shop. For the first column, we put shopid from the index of susp_order.

In [113]:
fraud_shop = pd.DataFrame(susp_order.index)

In [114]:
fraud_shop

,shopid
0,10159
1,10208
2,10287
3,10402
4,10536
...,...
823,209326384
824,210197928
825,211245313
826,212089630


Let's put another column inside fraud_shop table which comprise of fraud_buyer list. The column will be called userid.

In [116]:
fraud_shop['userid'] = fraud_buyer

In [117]:
fraud_shop

,shopid,userid
0,10159,62775725&214988798
1,10208,0
2,10287,74488711&198097381&214226569
3,10402,77819
4,10536,672345
...,...,...
823,209326384,0
824,210197928,52867898
825,211245313,0
826,212089630,0


We then assign shopid as the index of fraud_shop table.

In [118]:
fraud_shop.index = fraud_shop.shopid

In [119]:
fraud_shop

,shopid,userid
shopid,,
10159,10159,62775725&214988798
10208,10208,0
10287,10287,74488711&198097381&214226569
10402,10402,77819
10536,10536,672345
...,...,...
209326384,209326384,0
210197928,210197928,52867898
211245313,211245313,0


In [121]:
ls_order.suspicious_buyer.to_frame()

,suspicious_buyer
shopid,
10100,0
10151,0
10159,"[62775725, 214988798]"
10208,[11288307]
10214,0
...,...
211245313,[46875705]
211892380,0
212089630,[105664365]


The index of fraud_shop is made based on the range of fraud_shop rows. It has 828 rows, so the index will be ranged from 0 until 827.

In [122]:
fraud_shop.index = range(fraud_shop.shape[0])

In [123]:
fraud_shop

,shopid,userid
0,10159,62775725&214988798
1,10208,0
2,10287,74488711&198097381&214226569
3,10402,77819
4,10536,672345
...,...,...
823,209326384,0
824,210197928,52867898
825,211245313,0
826,212089630,0


Now, we go back to our original table and create new table out of it. First, shopid is grouped together, then list function is applied toward orderid column, which will be converted into dataframe. The new table will be stored in new variable called ls_order. We then add other two columns, which are userid and event_time which also be stored in a list form. Then from the columns that we have could know the number of order and the number of unique buyer. We use length function for the former and nunique function for the latter. Notice that this is exactly the same process as we did previously in first part of this notebook. Because we want to maintain the structure and number of rows of original table, but with the result in the form of userid listed as suspicious as we have gathered previously.  

In [124]:
ls_order = order_brush.groupby("shopid")["orderid"].apply(list).to_frame()
ls_order["userid"] = order_brush.groupby("shopid")["userid"].apply(list)
ls_order["event_time"] = order_brush.groupby("shopid")["event_time"].apply(list)

ls_order["num_order"] = list(map(lambda x: len(x), ls_order.orderid))

# count number of unique buyers for each shop

ls_order["num_unique_buyer"] = order_brush.groupby("shopid")["userid"].nunique()

In [125]:
ls_order = ls_order.userid.to_frame()

In [126]:
ls_order.userid = 0

In [127]:
ls_order.loc[fraud_shop.shopid]

,userid
shopid,
10159,0
10208,0
10287,0
10402,0
10536,0
...,...
209326384,0
210197928,0
211245313,0


The table fraud_shop is formed comprised of suspected shopid, with index exactly the same as shopid.

In [128]:
fraud_shop.index = fraud_shop.shopid

In [139]:
fraud_shop

,shopid,userid
shopid,,
10159,10159,62775725&214988798
10208,10208,0
10287,10287,74488711&198097381&214226569
10402,10402,77819
10536,10536,672345
...,...,...
209326384,209326384,0
210197928,210197928,52867898
211245313,211245313,0


We arrive to the last step of this solution which is creating table for submission so that it will consist of 18770 rows with two columns, shopid and userid. For the values, we extract the fraud_shop values which is the values that we have collected and consist of several number of suspicious userid that most likely conduct order brushing. We do the loop as shown below.

In [129]:
for shop in fraud_shop.shopid:
    ls_order.loc[shop] = fraud_shop.loc[shop]

Here is how final table will look like.

In [130]:
ls_order

,userid
shopid,
10009,0
10051,0
10061,0
10084,0
10100,0
...,...
214662358,0
214949521,0
214964814,0


After number of looping and list manipulating, we get this table at the end. Although it will not resulted in perfect score (in this case, I get 0.98% against the groud truth table provided by Shopee), it will be sufficient for this solution. We could always improve the accuracy by adding some methodologies and other things. I hope you could learn from this challenge as much as I am. Cheers!